In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from preprocessing import LABELS
from preprocessing import *
from functools import partial

from typing import Iterable, Any
from itertools import product

from time import time
import numpy as np
import pandas as pd
import random
import zipfile
from glob import glob
import os

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.autograph.set_verbosity(2)


2022-12-14 21:02:52.808729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 21:02:52.908266: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-14 21:02:52.912807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 21:02:52.912823: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
# !ls /datasets/minispeachcommands
# !unzip -q /datasets/minispeechcommands/msc-train.zip
# !unzip -q /datasets/minispeechcommands/msc-val.zip
# !unzip -q /datasets/minispeechcommands/msc-test.zip

In [4]:
go_stop_train_ds_pure = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
go_stop_val_ds_pure = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
go_stop_test_ds_pure = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])

num_train_files = len(go_stop_train_ds_pure)
num_val_files = len(go_stop_val_ds_pure)
num_test_files = len(go_stop_test_ds_pure)

print('Training set size:', num_train_files)
print('Validation set size:', num_val_files)
print('Test set size:', num_test_files)

go_stop_label = [LABELS[1], LABELS[5]]
print(go_stop_label)

2022-12-14 21:02:57.937932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-14 21:02:57.937957: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-14 21:02:57.937971: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-5c1936eb-da68-471b-86ff-fc150fc4e316): /proc/driver/nvidia/version does not exist
2022-12-14 21:02:57.938239: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Training set size: 1600
Validation set size: 200
Test set size: 

In [5]:
global get_frozen_spectrogram

def get_mfcc_and_label(filename, downsampling_rate, frame_length_in_s, frame_step_in_s, num_mel_bins, lower_frequency, upper_frequency, num_coefficients):
    mfccs, label = get_mfccs(filename, downsampling_rate, frame_length_in_s, frame_step_in_s, num_mel_bins, lower_frequency, upper_frequency, num_coefficients)
    
    return mfccs, label

def preprocess(filename):
    signal, label = get_frozen_spectrogram(filename)

    signal.set_shape(SHAPE)
    signal = tf.expand_dims(signal, -1)
    signal = tf.image.resize(signal, [32,32])
    # we put SHAPE instead of [32,32] since for testing part(with loading tflite) input must be [49,40,1]
    # so input for creating model is  [49,40,1] not [32,32,1]


    label_id = tf.argmax(label == go_stop_label)

    return signal, label_id

In [6]:
def test_acc_latency(interpreter, input_details, output_details, filenames, PREPROCESSING_ARGS):

    avg_preprocessing_latency = 0
    avg_model_latency = 0
    latencies = []
    accuracy = 0.0

    downsampling_rate = PREPROCESSING_ARGS['downsampling_rate']
    sampling_rate_int64 = tf.cast(downsampling_rate, tf.int64)
    frame_length = int(downsampling_rate * PREPROCESSING_ARGS['frame_length_in_s'])
    frame_step = int(downsampling_rate * PREPROCESSING_ARGS['frame_step_in_s'])
    num_spectrogram_bins = frame_length // 2 + 1
    num_mel_bins = PREPROCESSING_ARGS['num_mel_bins']
    lower_frequency = PREPROCESSING_ARGS['lower_frequency']
    upper_frequency = PREPROCESSING_ARGS['upper_frequency']

    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=num_mel_bins,
        num_spectrogram_bins=num_spectrogram_bins,
        sample_rate=downsampling_rate,
        lower_edge_hertz=lower_frequency,
        upper_edge_hertz=upper_frequency
    )

    print('filename len:', len(filenames))
    for filename in filenames:
        
        audio_binary = tf.io.read_file(filename)

        # NEED ONLY FOR TESTING
        path_parts = tf.strings.split(filename, '/')
        path_end = path_parts[-1]
        file_parts = tf.strings.split(path_end, '_')
        true_label = file_parts[0]
        true_label = true_label.numpy().decode()

        # PRE-PROCESSING (LOG-MEL SPECTROGRAM)
        start_preprocess = time()
        audio, sampling_rate = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio)

        zero_padding = tf.zeros(sampling_rate - tf.shape(audio), dtype=tf.float32)
        audio_padded = tf.concat([audio, zero_padding], axis=0)

        if downsampling_rate != sampling_rate:
            audio_padded = tfio.audio.resample(audio_padded, sampling_rate_int64, downsampling_rate)

        stft = tf.signal.stft(
            audio_padded,
            frame_length=frame_length,
            frame_step=frame_step,
            fft_length=frame_length
        )
        spectrogram = tf.abs(stft)

        mel_spectrogram = tf.matmul(spectrogram, linear_to_mel_weight_matrix)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)

        mfccs = tf.expand_dims(mfccs, 0)  # batch axis
        mfccs = tf.expand_dims(mfccs, -1)  # channel axis
        mfccs = tf.image.resize(mfccs, [32,32])

        end_preprocess = time()

        interpreter.set_tensor(input_details[0]['index'], mfccs)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        end_inference = time()

        top_index = np.argmax(output[0])
        predicted_label = go_stop_label[top_index]

        accuracy += true_label == predicted_label
        avg_preprocessing_latency += (end_preprocess - start_preprocess)
        avg_model_latency += (end_inference - end_preprocess)
        latencies.append(end_inference - start_preprocess)

    accuracy /= len(filenames)
    avg_preprocessing_latency /= len(filenames)
    avg_model_latency /= len(filenames)
    median_total_latency = np.median(latencies)

    return accuracy, avg_preprocessing_latency, avg_model_latency, median_total_latency


In [7]:
def simulator(downsampling_rate,frame_length_in_s, num_mel_bins, lower_frequency, upper_frequency,
                batch_size, initial_learning_rate, end_learning_rate, epochs, model_filter_param, alpha_param, final_sparsity_param):

    global get_frozen_spectrogram, SHAPE, alpha, final_sparsity, model_filter

    model_filter = model_filter_param
    alpha = alpha_param
    final_sparsity = final_sparsity_param
    frame_step_in_s_list = [frame_length_in_s]
    num_coefficients = num_mel_bins

    # **************************Model introducing

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
    metrics = [tf.metrics.SparseCategoricalAccuracy()]
    callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

    model_pure = tf.keras.Sequential([
        tf.keras.layers.Input(shape=[32,32, 1]),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(units=len(go_stop_label)),
        tf.keras.layers.Softmax()
    ])



    model_DS_CNN = tf.keras.Sequential([
        tf.keras.layers.Input(shape=[32,32, 1]),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[1, 1], strides=[1, 1], use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[1, 1], strides=[1, 1], use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(units=len(go_stop_label)),
        tf.keras.layers.Softmax()
    ])

    model_Width_Scaling = tf.keras.Sequential([
        tf.keras.layers.Input(shape=[32,32, 1]),
        tf.keras.layers.Conv2D(filters=int(model_filter * alpha), kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=int(model_filter * alpha), kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=int(model_filter * alpha), kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(units=len(go_stop_label)),
        tf.keras.layers.Softmax()
    ])


    model_Width_Pruning = tf.keras.Sequential([
        tf.keras.layers.Input(shape=[32,32, 1]),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=model_filter, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(units=len(go_stop_label)),
        tf.keras.layers.Softmax()
    ])



    for frame_step_in_s in frame_step_in_s_list:
        
        # **************************Copy the original data as local variable
        go_stop_train_ds = go_stop_train_ds_pure
        go_stop_val_ds = go_stop_val_ds_pure
        go_stop_test_ds = go_stop_test_ds_pure
        go_stop_test_ds_tflit = go_stop_test_ds_pure

        # **************************Definition of parameters
        PREPROCESSING_ARGS = {
            'downsampling_rate': downsampling_rate,
            'frame_length_in_s': frame_length_in_s,
            'frame_step_in_s': frame_step_in_s,
            'num_mel_bins': num_mel_bins,
            'num_coefficients': num_coefficients,
            'lower_frequency': lower_frequency,
            'upper_frequency': upper_frequency,
            
        }

        TRAINING_ARGS = {
            'batch_size': batch_size,
            'initial_learning_rate': initial_learning_rate,
            'end_learning_rate': end_learning_rate,
            'epochs': epochs
        }
        
        # **************************Getting mcff 
        global get_frozen_spectrogram, SHAPE

        get_frozen_spectrogram = partial(get_mfcc_and_label, **PREPROCESSING_ARGS)

        for mfcc, label in go_stop_train_ds.map(get_frozen_spectrogram).take(1):
            mfcc_sample = mfcc
            SHAPE = mfcc[:num_coefficients].shape
            print('SHAPE = ', SHAPE)
            
        
        
        # **************************Rezising and batching
        batch_size = batch_size
        epochs = epochs

        go_stop_train_ds = go_stop_train_ds.map(preprocess).batch(batch_size).cache()
        go_stop_val_ds = go_stop_val_ds.map(preprocess).batch(batch_size)
        go_stop_test_ds = go_stop_test_ds.map(preprocess).batch(batch_size)

        for example_batch, example_labels in go_stop_train_ds.take(1):
            print('Batch Shape:', example_batch.shape)
            print('Data Shape:', example_batch.shape[1:])
            print('Labels:', example_labels)
        

        # **************************Model definition
        # Preparation of optimizer
        linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=initial_learning_rate,
            end_learning_rate=end_learning_rate,
            decay_steps=len(go_stop_train_ds) * epochs,
        )
        optimizer = tf.optimizers.Adam(learning_rate=linear_decay)


        prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

        begin_step = int(len(go_stop_train_ds_pure) * epochs * 0.2)
        end_step = int(len(go_stop_train_ds_pure) * epochs)

        # Preparation of pruning model
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
                initial_sparsity=0.20,
                final_sparsity=final_sparsity,
                begin_step=begin_step,
                end_step=end_step
            )
        }


        # **************************Pick a model to compile
        model_list = [model_pure, model_DS_CNN, model_Width_Scaling]
        # model = model_list[2]
        model_for_pruning = prune_low_magnitude(model_Width_Scaling, **pruning_params)  # changeeeeeeeeeeeeeeeeeeeeeee

        # model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics) # changeeeeeeeeeeeeeeeeeeeeeee

        # **************************Start traning
        # history = model.fit(go_stop_train_ds, epochs=epochs, validation_data=go_stop_val_ds)
        history = model_for_pruning.fit(go_stop_train_ds, epochs=epochs, validation_data=go_stop_val_ds, callbacks=callbacks) # changeeeeeeeeeeeeeeeeeeeeeee
        
        # **************************Results of training
        training_loss = history.history['loss'][-1]
        training_accuracy = history.history['sparse_categorical_accuracy'][-1]
        val_loss = history.history['val_loss'][-1]
        val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]
        
        # print(f'Model name is: {model}')
        print(f'Training Loss: {training_loss:.4f}')
        print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
        print()
        print(f'Validation Loss: {val_loss:.4f}')
        print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
        print()
        
        # **************************Model evaluation without saving and latency
        # test_loss, test_accuracy = model.evaluate(go_stop_test_ds)
        test_loss, test_accuracy = model_for_pruning.evaluate(go_stop_test_ds) # changeeeeeeeeeeeeeeeeeeeeeee

        print(f'Test Loss: {test_loss:.4f}')
        print(f'Test Accuracy: {test_accuracy*100.:.2f}%')
        
        
        # **************************Model saving as tflite
        timestamp = int(time())

        saved_model_dir = f'./team10_final_8000/saved_models_grid_search/{timestamp}'
        if not os.path.exists(saved_model_dir):
            os.makedirs(saved_model_dir)
        
        # model.save(saved_model_dir)
        model_for_pruning.save(saved_model_dir) # changeeeeeeeeeeeeeeeeeeeeeee

        converter = tf.lite.TFLiteConverter.from_saved_model(f'./team10_final_8000/saved_models_grid_search/{timestamp}')
        tflite_model = converter.convert()

        tflite_models_dir = './team10_final_8000/saved_models_grid_search_tflite_models'
        if not os.path.exists(tflite_models_dir):
            os.makedirs(tflite_models_dir)
        
        tflite_model_name = os.path.join(tflite_models_dir, f'{timestamp}.tflite')
        tflite_model_name

        with open(tflite_model_name, 'wb') as fp:
            fp.write(tflite_model)

        with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
            f.write(tflite_model_name)

        zipped_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024.0


        # **************************Model evaluation with loading tflite and latency
        interpreter = tf.lite.Interpreter(model_path=f'./team10_final_8000/saved_models_grid_search_tflite_models/{timestamp}.tflite')
        interpreter.allocate_tensors()

        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        print("Number of inputs:", len(input_details))
        print("Number of outputs:", len(output_details))
        print("Input name:", input_details[0]['name'])
        print("Input shape:", input_details[0]['shape'])
        print("Output name:", output_details[0]['name'])
        print("Output shape:", output_details[0]['shape'])

        accuracy, avg_preprocessing_latency, avg_model_latency, median_total_latency = test_acc_latency(interpreter, input_details, output_details, go_stop_test_ds_tflit, PREPROCESSING_ARGS)
        model_size = os.path.getsize(f'./team10_final_8000/saved_models_grid_search_tflite_models/{timestamp}.tflite')
        

        print(f'Accuracy: {100 * accuracy:.3f}%')
        print(f'Model size: {model_size / 2 ** 10:.1f}KB')
        print(f'Zipped tflite size: {zipped_size:.3f} KB')
        print(f'Preprocessing Latency: {1000 * avg_preprocessing_latency:.1f}ms')
        print(f'Model Latency: {1000 * avg_model_latency:.1f}ms')
        print(f'Total Latency: {1000 * median_total_latency:.1f}ms')

        output_dict = {
            'timestamp_model_name': timestamp,
            **PREPROCESSING_ARGS,
            **TRAINING_ARGS,
            'test_accuracy': test_accuracy,
            'accuracy': accuracy,
            'avg_preprocessing_latency': avg_preprocessing_latency,
            'avg_model_latency': avg_model_latency,
            'median_total_latency': median_total_latency,
            'Model size': model_size / 2 ** 10,
            'Zipped tflite size': zipped_size,
            'model_filter': model_filter,
            'alpha': alpha,
            'final_sparsity': final_sparsity
        }

        df = pd.DataFrame([output_dict])

        saved_out_put_dir = './team10_final_8000/out_put_dict_grid_search'
        if not os.path.exists(saved_out_put_dir):
            os.makedirs(saved_out_put_dir)

        # output_path = f'./team10_testing/out_put_dict_grid_search/spectrogram_results_width_scaling.csv'
        output_path = f'./team10_final_8000/out_put_dict_grid_search/spectrogram_results_pruning_width_scaling.csv'
        df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)



In [8]:
    # 'downsampling_rate': [16000], # [8000, 16000]
    # 'frame_length_in_s': [0.016], # [0.08, 0.016, 0.04, 0.08]
    # # 'frame_step_in_s': [0.02], # using [0.25, 0.5 ,1] as coefficient of frame_length_in_s
    # 'num_mel_bins': [20], # [20, 40]
    # 'lower_frequency': [20],
    # 'upper_frequency': [8000], # [4000, 8000]
    # # 'num_coefficients': [20], # using [0.5 ,1] as coefficient of num_mel_bins
    # 'batch_size': [20], # [10, 20, 30]
    # 'initial_learning_rate': [0.01],
    # 'end_learning_rate': [1.e-5],
    # 'epochs': [20], # [10, 20, 30]
    # 'model_filter':[64, 128, 256],
    # 'alpha': [0.05, 0.1, 0.2, 0.3],
    # # 'alpha_DS': [0.05],
    # 'final_sparsity': [0.5, 0.6, 0.7, 0.8],
    # # 'final_sparsity_DS': [0.6],

PREPROCESSING_TRAINING_ARGS = {
    'downsampling_rate': [16000],
    'frame_length_in_s': [0.016],
    # 'frame_step_in_s': [0.02],
    'num_mel_bins': [20],
    'lower_frequency': [20],
    'upper_frequency': [8000],
    # 'num_coefficients': [20],
    'batch_size': [20],
    'initial_learning_rate': [0.01],
    'end_learning_rate': [1.e-5],
    'epochs': [20],
    'model_filter':[64, 128, 256],
    'alpha': [0.05, 0.1, 0.2, 0.3],
    # 'alpha_DS': [0.05],
    'final_sparsity': [0.5, 0.6, 0.7, 0.8],
    # 'final_sparsity_DS': [0.6],

}

def grid_parameters(parameters: dict[str, Iterable[Any]]) -> Iterable[dict[str, Any]]:
    for params in product(*parameters.values()):
        yield dict(zip(parameters.keys(), params))

In [9]:
for parameters in grid_parameters(PREPROCESSING_TRAINING_ARGS):
    print('parameters are: ', parameters)
    simulator(*parameters.values())
    # print('parameters are: ', parameters)
    # break

Output name: StatefulPartitionedCall:0
Output shape: [1 2]
filename len: 200
Accuracy: 100.000%
Model size: 54.6KB
Zipped tflite size: 45.621 KB
Preprocessing Latency: 5.1ms
Model Latency: 0.2ms
Total Latency: 5.1ms
parameters are:  {'downsampling_rate': 16000, 'frame_length_in_s': 0.016, 'num_mel_bins': 20, 'lower_frequency': 20, 'upper_frequency': 8000, 'batch_size': 20, 'initial_learning_rate': 0.01, 'end_learning_rate': 1e-05, 'epochs': 20, 'model_filter': 256, 'alpha': 0.1, 'final_sparsity': 0.7}
2022-12-14 21:46:06.916622: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-14 21:46:06.918090: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-14 21:46:06.918251: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
SHAPE =  (20, 20)
2022-12-14 21:46:07.49

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>